In [ ]:
import db
from map import Map, Marker

from ipyleaflet import (
    Popup,
    LayerGroup,
    WidgetControl,
    AwesomeIcon,
    AntPath,
)
from ipywidgets import HTML, IntSlider, Button, Layout

In [ ]:
top_trips_query = """
    with rides as (
        select start_station_id, end_station_id, count(*) as num_rides
        
        from divvy_station_trips e 
        
        group by 1,2
    )
    
    select r.*
    
    , start_s.name as start_name
    , start_s.lat as start_lat
    , start_s.lon as start_lon
    
    
    , end_s.name as end_name
    , end_s.lat as end_lat
    , end_s.lon as end_lon
    
    from rides r
    
    left join divvy_stations start_s on start_s.station_id = r.start_station_id
    left join divvy_stations end_s on end_s.station_id = r.end_station_id
    
    order by num_rides desc
"""
db.query(top_trips_query)

In [ ]:
m = Map((41.8, -87.5))
m.m.layout = Layout(height="1000px")

map_markers = LayerGroup()
m.add(map_markers)

slider = IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description="Limit:",
    style=dict(description_width="100px"),
    continuous_update=False,
)
m.add(WidgetControl(widget=slider, position="topright"))

display(m)

In [ ]:
def draw_markers(change):
    map_markers.clear_layers()

    top_trips = db.query(f"{top_trips_query} limit {int(change['new'])}").fetchall()

    for row in top_trips:
        m.markers.append(Marker((row[4], row[5]), f"{row[3]}", "bicycle"))

        ant_path = AntPath(
            locations=[[row[4], row[5]], [row[7], row[8]]],
            dash_array=[1, 20],
            delay=3000,
            color="#7590ba",
            pulse_color="#3f6fba",
        )

        map_markers.add(ant_path)

        m.markers.append(Marker((row[7], row[8]), f"{row[6]}", "bicycle"))

    m.redraw()


draw_markers({"new": slider.value})
slider.observe(draw_markers, names="value")

In [ ]:
out_of_bounds = db.query(
    """
with buffered as (
    select id, st_buffer(geom, 0.005) as geom
    from boundaries
    where id = 'IL'
)

select lat, lon, e.event_at
from divvy_station_events e
left join buffered b on st_contains(b.geom, st_flipcoordinates(e.geom))
where b.id is null
"""
)
out_of_bounds

In [ ]:
boundaries = db.query(
    """
select id, st_asgeojson(st_buffer(geom, 0.005)) as geom
from boundaries
where id = 'IL' 
"""
)

In [ ]:
from ipyleaflet import GeoJSON
import json

m = Map((41.85, -87.5))
for row in out_of_bounds.fetchall():
    m.markers.append(Marker((row[0], row[1]), str(row[2]), "bicycle"))
m

for row in boundaries.fetchall():
    geo_json = GeoJSON(data=json.loads(row[1]), style={"color": "blue", "weight": 1})
    m.add(geo_json)
m